In [ ]:
# モデルの保存と復元
# モデルをほかのプログラムで使用したり、比較するために使う。
# 訓練中に適宜保存することでクラッシュ後のリラン対応も行える


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
import os

# reset_graph用メソッド
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
# m : housing.data.shape の行が設定される　20640
# n : housing.data.shape の列が設定される  8

# housing.data : データの中身を見れる(多次元配列)
# hoousing.data.shape : 配列の定義を見れる ここでは(20640,8) 20640インスタンス、特徴量8個

In [3]:
# 勾配降下法はスケーリングが重要なのでまずはスケーリングを行う。手法はお任せ。ここではscikit-learnで実施する
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data) # housing.data を指定してスケーリングを行う
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
# np.ones((m, 1)) : [1.]の20640個の配列
# np.c_ : 行列の連結
# np.c_[np.ones((m, 1)), housing.data] : [1., housing.dataの特徴量(8個)] の20640インスタンスのnumpy配列が出来上がる
# 先頭にバイアスとして　１　が設定されるnumpy配列ができているということ

In [7]:
reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
# random_uniform : 与えた形状と範囲に基づいてランダム値を格納するテンソルを生成する
# tf.random_uniform([n + 1, 1]) では (9,1)のnumpy配列　ここでは操作名"theata" として(9,1)のnumpy配列 最小値-1,最大値1 で作成
# theataが重みをもった特徴量ベクトルになる

#y_pred = tf.matmul(X, theta, name="predictions")
# tf.matmul : 行列積  ここでは訓練データ(20640,9)と(9,1)の行列積となる python 3.5 からは以下の書き方がある
# ここで行列積を出しているのは入力となるnumpy配列の特徴量ベクトルと重み更新のためのtheata特徴量ベクトルをかけることにより予測値を出すため
#python3.5からの簡単な記載
y_pred = X @ theta
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
# 構築フェーズの最後にSaverノードを作成する
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "C:/Users/leuyx/Documents/ml/09_model/my_model.ckpt") #　モデルを保存したいタイミングでsave()メソッドを実行する
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "C:/Users/leuyx/Documents/ml/09_model/my_model_final.ckpt") # 最終的に作成されたモデルの保存

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474


In [8]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

In [10]:
# モデルの復元は構築フェーズの最後でSaverを作っておき、実行フェーズにinitノードを使うのではなく、restoreメソッドを呼んで復元させる
with tf.Session() as sess:
    saver.restore(sess, "C:/Users/leuyx/Documents/ml/09_model/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from C:/Users/leuyx/Documents/ml/09_model/my_model_final.ckpt


In [11]:
np.allclose(best_theta, best_theta_restored) # これはリストア前後の配列が同じか比較している

True

In [ ]:
# この機能を使うことで事前学習済みのモデルを使うことができる